In [ ]:
%run DOVSOutages.ipynb
%run AMINonVee.ipynb
%run AMINonVeeSQL.ipynb
%run AMINonVeeCircuitSQL.ipynb
%run MeterPremise.ipynb

In [4]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import itertools
import copy
import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Plot_Box_sns
import GrubbsTest

# ------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------

In [ ]:
def get_std_date_ranges_and_tags_templates(division_type='quarterly'):
    # This is a very simple function, and at this point division_type can only be 
    # biannual or quarterly
    assert(division_type=='quarterly' or division_type=='biannual')
    if division_type=='quarterly':
        date_ranges_and_tags_templates = [
            (['{}-01-01', '{}-03-31'], '{}_q1'), 
            (['{}-04-01', '{}-06-30'], '{}_q2'), 
            (['{}-07-01', '{}-09-30'], '{}_q3'), 
            (['{}-10-01', '{}-12-31'], '{}_q4')
        ]
    elif division_type=='biannual':
        date_ranges_and_tags_templates = [
            (['{}-01-01', '{}-06-30'], '{}_h1'), 
            (['{}-07-01', '{}-12-31'], '{}_h2')
        ]
    else:
        assert(0)    
    return date_ranges_and_tags_templates

def inject_year_into_date_ranges_and_tags_templates(date_ranges_and_tags_templates, year):
    date_ranges_and_tags = []
    for period_template in date_ranges_and_tags_templates:
        dt_rng_min_tmplt = period_template[0][0]
        dt_rng_max_tmplt = period_template[0][1]
        tag_tmplt = period_template[1]
        #--------------------
        date_ranges_and_tags.append((
            [dt_rng_min_tmplt.format(year), dt_rng_max_tmplt.format(year)], 
            tag_tmplt.format(year)
        ))
    return date_ranges_and_tags

# ------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------

In [ ]:
#TODO Method to split up MP into multiple when number of trsf_pole_nbs is large

In [ ]:
conn_aws = Utilities.get_athena_prod_aws_connection()

# ------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------

In [ ]:
# cols_of_interest_usage = ['serialnumber', 'starttimeperiod', 'endtimeperiod', 'aep_endtime_utc', 
#                           'timezoneoffset', 'aep_derived_uom', 'aep_srvc_qlty_idntfr', 'value', 
#                           'aep_opco', 'aep_usage_dt']

cols_of_interest_usage = ['serialnumber', 'starttimeperiod', 'endtimeperiod', 'aep_endtime_utc', 'timezoneoffset',
                          'aep_derived_uom', 'aep_srvc_qlty_idntfr', 'value', 'aep_usage_dt']

# cols_of_interest_met_prem = ['mfr_devc_ser_nbr', 'mtr_stat_cd_desc', 'first_in_srvc_dt', 'phys_inst_dt',
#                              'inst_tod_cd', 'inst_ts', 'last_fld_test_date', 'longitude', 'latitude',
#                              'state_cd',  'prem_nb', 'prem_stat_cd', 'frst_turn_on_dt', 'hsng_ctgy_cd',
#                              'srvc_pole_nb', 'trsf_pole_nb', 'curr_bill_acct_id', 'curr_bill_acct_nb',
#                              'annual_kwh', 'annual_max_dmnd', 'latitude_nb', 'longitude_nb', 'station_nb',  
#                              'xfmr_nb']

cols_of_interest_met_prem = ['mfr_devc_ser_nbr', 'prem_nb', 'srvc_pole_nb', 
                             'trsf_pole_nb', 'annual_kwh', 'station_nb', 'xfmr_nb']

In [ ]:
#------------- ORIGINAL BATCH --------------------------------------------------------------------------
# dev_ser_nbrs should work whether elements are strings or ints
# outg_rec_nb = 11770899
# dev_ser_nbrs = [883024464, 883024356, 883409944, 682540032, 883409946, 883409945, 883409947, 881902429]

# outg_rec_nb = 11751094
# dev_ser_nbrs = [880687438, 880713197, 880687439, 880693917, 880693914, 880693921]

# outg_rec_nb = 11738944
# dev_ser_nbrs = [882733737, 882731886, 882454685, 882733663, 882733768, 882733694, 882731883, 882733767]

#------------- NEW BATCH --------------------------------------------------------------------------
# outg_rec_nb = 11496239
# dev_ser_nbrs = [883464195, 883463751, 883464198, 883463775, 883463138, 883463776]

# outg_rec_nb = 11818003
# dev_ser_nbrs = [882991118, 884393033, 883628907, 883034206, 883034208, 878801985]

# outg_rec_nb = 11833199
# dev_ser_nbrs = [876722512, 876114812, 876722442, 876114813, 876722510, 876114810]

outg_rec_nb = 11851647
dev_ser_nbrs = [884390632, 880682782, 880320285, 880076534, 880320207, 889926007, 884390655, 880076535]

# outg_rec_nb = 12022118
# dev_ser_nbrs = [555666226, 655187440, 655187438, 555660251, 555660205]

In [ ]:
save_dir_base = os.path.join(Utilities.get_local_data_dir(), r'sample_outages_full_NEW')
save_csv = False

year = '2019'
date_range_idx = 0
#-----
date_ranges_and_tags_templates = date_ranges_and_tags_templates = get_std_date_ranges_and_tags_templates('quarterly')
date_ranges_and_tags = inject_year_into_date_ranges_and_tags_templates(date_ranges_and_tags_templates, year)
#-----
date_range = date_ranges_and_tags[date_range_idx][0]
save_time_tag = date_ranges_and_tags[date_range_idx][1]

assert(date_range[0][:4]==date_range[1][:4]==save_time_tag[:4])

#------------------------------------
outg_rec_nb = 11770899
dev_ser_nbrs = [883024464, 883024356, 883409944, 682540032, 883409946, 883409945, 883409947, 881902429]

In [ ]:
run_single=False
join_meter_premise=True
if run_single:
    if join_meter_premise:
        sql = AMINonVeeSQL.build_usg_join_mp_sql(cols_of_interest_usage, cols_of_interest_met_prem, 
                                                 aep_derived_uoms_and_idntfrs=None, 
                                                 kwh_and_vlt_only=False, 
                                                 build_usg_kwargs=dict(date_range=date_range, serial_numbers=dev_ser_nbrs, aep_opco='oh'), 
                                                 build_mp_kwargs=dict(serial_numbers=dev_ser_nbrs))
    else:
        sql = AMINonVeeSQL.build_sql_usg(cols_of_interest_usage, 
                                         aep_derived_uoms_and_idntfrs=None, 
                                         kwh_and_vlt_only=kwh_and_vlt_only, 
                                         serial_numbers=dev_ser_nbrs, 
                                         date_range=date_range, aep_opco='oh')
        sql = sql.get_sql_statement()
    print(sql)
    #--------------------------------------------------------------------
    df = pd.read_sql(sql, conn_aws)
    df = clm.remove_prepend_from_columns_in_df(df)
    #--------------------------------------------------------------------
    if save_csv:
        save_path = os.path.join(save_dir_base, f"outg_rec_nb_{outg_rec_nb}", f"outg_rec_nb_{outg_rec_nb}_{save_time_tag}.csv")
        df.to_csv(save_path, index=False)

In [ ]:
run_full = False
join_meter_premise=True
kwh_and_vlt_only=False
save_dir_base = os.path.join(Utilities.get_local_data_dir(), r'sample_outages_full_NEW')
save_csv = False
verbose=True
#------------------------------------
years = [2018, 2019, 2020, 2021]
date_ranges_and_tags_templates = date_ranges_and_tags_templates = get_std_date_ranges_and_tags_templates('quarterly')
#------------------------------------
outg_rec_nb = 11851647
dev_ser_nbrs = [884390632, 880682782, 880320285, 880076534, 880320207, 889926007, 884390655, 880076535]

In [ ]:
if run_full:
    for year in years:        
        date_ranges_and_tags = inject_year_into_date_ranges_and_tags_templates(date_ranges_and_tags_templates, year)
        for date_ranges_and_tag in date_ranges_and_tags:
            date_range = date_ranges_and_tag[0]
            save_time_tag = date_ranges_and_tag[1]
            assert(date_range[0][:4]==date_range[1][:4]==save_time_tag[:4])
            #--------------------------------------------------------------------
            if join_meter_premise:
                sql = AMINonVeeSQL.build_usg_join_mp_sql(cols_of_interest_usage, cols_of_interest_met_prem, serial_numbers=dev_ser_nbrs, 
                                                         aep_derived_uoms_and_idntfrs=None, 
                                                         kwh_and_vlt_only=kwh_and_vlt_only, 
                                                         build_usg_kwargs=dict(date_range=date_range, aep_opco='oh'))
            else:
                sql = AMINonVeeSQL.build_sql_usg(cols_of_interest_usage, 
                                                 aep_derived_uoms_and_idntfrs=None, 
                                                 kwh_and_vlt_only=kwh_and_vlt_only, 
                                                 serial_numbers=dev_ser_nbrs, 
                                                 date_range=date_range, aep_opco='oh')
                sql = sql.get_sql_statement()
            if verbose:
                print('*'*100)
                print(sql)
            #--------------------------------------------------------------------
            df = pd.read_sql(sql, conn_aws)
            df = clm.remove_prepend_from_columns_in_df(df)
            #--------------------------------------------------------------------
            if save_csv:
                save_dir = os.path.join(save_dir_base, f"outg_rec_nb_{outg_rec_nb}")
                if not os.path.exists(save_dir):
                    os.makedirs(save_dir)
                save_path = os.path.join(save_dir, f"outg_rec_nb_{outg_rec_nb}_{save_time_tag}.csv")
                print('Saving to:', save_path)
                df.to_csv(save_path, index=False)

# ------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------

# FINDING OTHERS ON THE CIRCUIT

In [ ]:
#TODO BETTER NAME
def run_and_write_multiple(
    conn_aws, save_dir_base, 
    years, date_ranges_and_tags_templates, 
    serial_numbers, 
    field_descs, agg_cols_and_types, groupby_cols, 
    aep_derived_uoms_and_idntfrs=None, 
    calculate_net_kwh=False, addtnl_build_net_kwh_kwargs=None, 
    stop_after_agg_1=False, groupby_xfmr_in_agg_1=True, 
    include_counts_including_null=True, 
    usg_alias='U', mp_alias='MP', agg_1_alias='AGG1', 
    addtnl_build_agg_1_sql_kwargs=None, 
    verbose=True, 
    **kwargs
):
    for year in years:
        date_ranges_and_tags = inject_year_into_date_ranges_and_tags_templates(date_ranges_and_tags_templates, year)
        for date_ranges_and_tag in date_ranges_and_tags:
            date_range = date_ranges_and_tag[0]
            save_time_tag = date_ranges_and_tag[1]
            assert(date_range[0][:4]==date_range[1][:4]==save_time_tag[:4])
            #--------------------------------------------------------------------
            if verbose:
                print('*'*100)
            sql = AMINonVeeCircuitSQL.build_sql_outage_others_on_circuit(
                conn_aws=conn_aws, 
                serial_numbers=serial_numbers, date_range=date_range, 
                field_descs=field_descs, agg_cols_and_types=agg_cols_and_types, groupby_cols=groupby_cols, 
                aep_derived_uoms_and_idntfrs=aep_derived_uoms_and_idntfrs, 
                calculate_net_kwh=calculate_net_kwh, addtnl_build_net_kwh_kwargs=addtnl_build_net_kwh_kwargs, 
                stop_after_agg_1=stop_after_agg_1, groupby_xfmr_in_agg_1=groupby_xfmr_in_agg_1,  
                include_counts_including_null=include_counts_including_null, 
                addtnl_build_agg_1_sql_kwargs=addtnl_build_agg_1_sql_kwargs, 
                verbose=verbose, 
                **kwargs
            )
            #--------------------------------------------------------------------
            df = pd.read_sql(sql, conn_aws)
            df = clm.remove_prepend_from_columns_in_df(df)
            #--------------------------------------------------------------------
            save_dir = os.path.join(save_dir_base, f"outg_rec_nb_{outg_rec_nb}")
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            save_path = os.path.join(save_dir, f"outg_rec_nb_{outg_rec_nb}_{save_time_tag}.csv")
            print('Saving to:', save_path)
            df.to_csv(save_path, index=False)

In [ ]:
cols_of_interest_usage_agg = ['starttimeperiod', 'endtimeperiod', 'aep_endtime_utc', 'timezoneoffset', 
                              'aep_derived_uom', 'aep_srvc_qlty_idntfr', 'value', 'aep_usage_dt']

serial_numbers = dev_ser_nbrs
date_range = ['2021-01-01', '2021-01-02']
field_descs=cols_of_interest_usage_agg
agg_cols_and_types = {'U.value':['sum', 'sq_sum', 'mean', 'std', 'count']}
groupby_cols = ['starttimeperiod', 'endtimeperiod', 'aep_endtime_utc', 'timezoneoffset', 
                'aep_derived_uom', 'aep_srvc_qlty_idntfr', 'aep_usage_dt']
try_to_split_col_strs = True

In [7]:
#-------------------------
# v1 
calculate_net_kwh = False
stop_after_agg_1 = True 
groupby_xfmr_in_agg_1 = False
save_dir_base = os.path.join(Utilities.get_local_data_dir(), r'sample_circuits', r'NewMethod')
#-------------------------
# v1 grouped by xfmr
# calculate_net_kwh = False
# stop_after_agg_1 = True 
# groupby_xfmr_in_agg_1 = True
# save_dir_base = os.path.join(Utilities.get_local_data_dir(), r'sample_circuits', r'NewMethod', r'GroupByXfmr')
#-------------------------
# v2 
# calculate_net_kwh = False
# stop_after_agg_1 = False
# groupby_xfmr_in_agg_1 = True
# save_dir_base = os.path.join(Utilities.get_local_data_dir(), r'sample_circuits', r'NewMethod', r'GroupByXfmr_v2')
#-------------------------
# v3
# calculate_net_kwh = True
# stop_after_agg_1 = False 
# groupby_xfmr_in_agg_1 = True
# save_dir_base = os.path.join(Utilities.get_local_data_dir(), r'sample_circuits', r'NewMethod', r'GroupByXfmr_v3_NET')
#--------------------------------------------------
#--------------------------------------------------

In [ ]:
run_full = False
run_single = False
save_csv = False
verbose=True
#------------------------------------
years = [2018, 2019, 2020, 2021]
date_ranges_and_tags_templates = get_std_date_ranges_and_tags_templates('quarterly')
#-----
#date_range_single = ['2019-01-01', '2019-03-31']; save_time_tag = '2019_q1'
#date_range_single = ['2019-04-01', '2019-06-30']; save_time_tag = '2019_q2'
#date_range_single = ['2019-07-01', '2019-09-30']; save_time_tag = '2019_q3'
date_range_single = ['2019-10-01', '2019-12-31']; save_time_tag = '2019_q4'
#------------------------------------
outg_rec_nb = 11738944
dev_ser_nbrs = [882733737, 882731886, 882454685, 882733663, 882733768, 882733694, 882731883, 882733767]

In [ ]:
if run_single:
    assert(date_range_single[0][:4]==date_range_single[1][:4]==save_time_tag[:4])
    sql = AMINonVeeCircuitSQL.build_sql_outage_others_on_circuit(
        conn_aws, 
        dev_ser_nbrs, date_range_single, 
        field_descs, agg_cols_and_types, groupby_cols, 
        aep_derived_uoms_and_idntfrs=None, 
        calculate_net_kwh=False, addtnl_build_net_kwh_kwargs=None, 
        stop_after_agg_1=True, groupby_xfmr_in_agg_1=groupby_xfmr_in_agg_1, 
        include_counts_including_null=True,  
        addtnl_build_agg_1_sql_kwargs=dict(try_to_split_col_strs=try_to_split_col_strs), 
        verbose=verbose
    )
    #-----
    df = pd.read_sql(sql, conn_aws)
    df = clm.remove_prepend_from_columns_in_df(df)
    if save_csv:
        save_path = os.path.join(save_dir_base, f"outg_rec_nb_{outg_rec_nb}", f"outg_rec_nb_{outg_rec_nb}_{save_time_tag}.csv")
        df.to_csv(save_path, index=False)    

In [ ]:
if run_full:
    run_and_write_multiple(
        conn_aws=conn_aws, save_dir_base=save_dir_base, 
        years=years, date_ranges_and_tags_templates=date_ranges_and_tags_templates, 
        serial_numbers=dev_ser_nbrs, 
        field_descs=field_descs, agg_cols_and_types=agg_cols_and_types, groupby_cols=groupby_cols, 
        aep_derived_uoms_and_idntfrs=None, 
        calculate_net_kwh=calculate_net_kwh, addtnl_build_net_kwh_kwargs=None, 
        stop_after_agg_1=stop_after_agg_1, groupby_xfmr_in_agg_1=groupby_xfmr_in_agg_1,  
        include_counts_including_null=True, 
        addtnl_build_agg_1_sql_kwargs=dict(try_to_split_col_strs=try_to_split_col_strs), 
        verbose=verbose
    )

# TODO build_sql_outage_others_on_circuit_3b not fully functional!!!!!
- I should basically be able to achieve this now by simply switching allow_kwh_in_additional_derived_uoms to True in 
AMINonVeeSQL.build_net_kwh_usage_sql_statement

In [ ]:
def build_sql_outage_others_on_circuit_3b(conn_aws, dev_ser_nbrs, date_range, verbose=True):
    # Return SQL statement to build aggregate of all OTHER meters on the circuit (i.e., excluding those from dev_ser_nbrs)
    # ****************************************************
    # Step 1: From the serial numbers given in dev_ser_nbrs, first find the circuit information
    #         and ensure all of the listed serial numbers are on the same circuit
    # Step 2: Given the circuit information, find all transformers on the circuit.
    #         This information is needed because not all meters contain circuit data, and without the transformer
    #           numbers on the circuit these meters would be left out.
    #         In the final query, meters on the circuit will be found which either have the correct circuit
    #           information OR the correct transformer number.
    #           NOTE: To be 100% correct, it should probably be meters which have the correct circuit information
    #                 OR no circuit infomration AND the correct transformer number.
    # Step 3: Put it all together.  Return SQL statement to build aggregate of all OTHER meters on the circuit 
    #           (i.e., excluding those from dev_ser_nbrs)
    # ****************************************************
    
    # ********************** Step 1 **********************
    mp_where_str_1 = "mfr_devc_ser_nbr IN ({})".format(','.join(["'{}'".format(x) for x in dev_ser_nbrs]))
    sql_mp_1 = (
    """
    SELECT DISTINCT circuit_nb,circuit_nm,station_nb,station_nm
    FROM default.meter_premise
    WHERE {}
    AND circuit_nb IS NOT NULL AND circuit_nb <> ''
    AND circuit_nm IS NOT NULL AND circuit_nm <> ''
    AND station_nb IS NOT NULL AND station_nb <> ''
    AND station_nm IS NOT NULL AND station_nm <> ''
    """
    ).format(mp_where_str_1)
    #--------------------
    df_mp_1 = pd.read_sql(sql_mp_1, conn_aws)
    assert(df_mp_1.shape[0]==1)
    #--------------------
    circuit_nb = df_mp_1.iloc[0]['circuit_nb']
    circuit_nm = df_mp_1.iloc[0]['circuit_nm']
    station_nb = df_mp_1.iloc[0]['station_nb']
    station_nm = df_mp_1.iloc[0]['station_nm']

    # ********************** Step 2 **********************
    mp_where_str_2 = (
    f"""
    circuit_nb = '{circuit_nb}'
    AND circuit_nm = '{circuit_nm}'
    AND station_nb = '{station_nb}'
    AND station_nm = '{station_nm}'
    """
    )

    sql_mp_2 = (
    """
    SELECT DISTINCT trsf_pole_nb
    FROM default.meter_premise
    WHERE {}
    """
    ).format(mp_where_str_2)
    #--------------------
    df_mp_2 = pd.read_sql(sql_mp_2, conn_aws)
    trsf_pole_nbs = df_mp_2['trsf_pole_nb'].tolist()

    # ********************** Step 3 **********************
    mp_where_str_3 = (
    """
    (circuit_nb = '{}'
    AND circuit_nm = '{}'
    AND station_nb = '{}'
    AND station_nm = '{}')
    OR trsf_pole_nb IN ({})
    """
    ).format(circuit_nb, circuit_nm, station_nb, station_nm, 
             ','.join(["'{}'".format(x) for x in trsf_pole_nbs]))   
    #--------------------  
    sql = (
    """
    WITH MP
    AS (
        SELECT mfr_devc_ser_nbr, trsf_pole_nb
        FROM default.meter_premise
        WHERE {0}
    ), 
    KWH_USG_DELREC_W_SIGNED_VAL AS (
        SELECT serialnumber, starttimeperiod, endtimeperiod, aep_endtime_utc, timezoneoffset, 
               aep_derived_uom, aep_srvc_qlty_idntfr, aep_usage_dt, value, IF(aep_srvc_qlty_idntfr='RECEIVED', -1*value, value) as signed_value
        FROM usage_nonvee.reading_ivl_nonvee
        WHERE aep_opco = 'oh'
        AND aep_derived_uom = 'KWH'
        AND aep_usage_dt BETWEEN {1} AND {2}    
    ), 
    USG_NET AS(
        SELECT serialnumber, starttimeperiod, endtimeperiod, aep_endtime_utc, timezoneoffset, 
               aep_derived_uom, aep_usage_dt, SUM(signed_value) as value_net
        FROM KWH_USG_DELREC_W_SIGNED_VAL
        GROUP BY serialnumber, starttimeperiod, endtimeperiod, aep_endtime_utc, timezoneoffset, 
                 aep_derived_uom, aep_usage_dt
    ), 
    USG_REC AS(
        SELECT serialnumber, starttimeperiod, endtimeperiod, aep_endtime_utc, timezoneoffset, 
               aep_derived_uom, aep_usage_dt, value as value_rec
        FROM KWH_USG_DELREC_W_SIGNED_VAL
        WHERE aep_srvc_qlty_idntfr='RECEIVED'
    ), 
    USG_DEL AS(
        SELECT serialnumber, starttimeperiod, endtimeperiod, aep_endtime_utc, timezoneoffset, 
               aep_derived_uom, aep_usage_dt, value as value_del
        FROM KWH_USG_DELREC_W_SIGNED_VAL
        WHERE aep_srvc_qlty_idntfr='DELIVERED'
    ), 
    USG_TOT AS(
        SELECT serialnumber, starttimeperiod, endtimeperiod, aep_endtime_utc, timezoneoffset, 
               aep_derived_uom, aep_usage_dt, value as value_tot
        FROM KWH_USG_DELREC_W_SIGNED_VAL
        WHERE aep_srvc_qlty_idntfr='TOTAL'
    ),    
    U AS (
        SELECT USG_NET.serialnumber, USG_NET.starttimeperiod, USG_NET.endtimeperiod, USG_NET.aep_endtime_utc, USG_NET.timezoneoffset, 
               USG_NET.aep_derived_uom, USG_NET.aep_usage_dt, USG_NET.value_net, USG_REC.value_rec, USG_DEL.value_del, USG_TOT.value_tot
        FROM USG_NET
        INNER JOIN USG_REC
        ON USG_NET.serialnumber=USG_REC.serialnumber AND USG_NET.starttimeperiod=USG_REC.starttimeperiod AND USG_NET.endtimeperiod=USG_REC.endtimeperiod AND USG_NET.aep_endtime_utc=USG_REC.aep_endtime_utc AND 
           USG_NET.timezoneoffset=USG_REC.timezoneoffset AND USG_NET.aep_derived_uom=USG_REC.aep_derived_uom AND USG_NET.aep_usage_dt=USG_REC.aep_usage_dt
        INNER JOIN USG_DEL
        ON USG_NET.serialnumber=USG_DEL.serialnumber AND USG_NET.starttimeperiod=USG_DEL.starttimeperiod AND USG_NET.endtimeperiod=USG_DEL.endtimeperiod AND USG_NET.aep_endtime_utc=USG_DEL.aep_endtime_utc AND 
           USG_NET.timezoneoffset=USG_DEL.timezoneoffset AND USG_NET.aep_derived_uom=USG_DEL.aep_derived_uom AND USG_NET.aep_usage_dt=USG_DEL.aep_usage_dt
        INNER JOIN USG_TOT
        ON USG_NET.serialnumber=USG_TOT.serialnumber AND USG_NET.starttimeperiod=USG_TOT.starttimeperiod AND USG_NET.endtimeperiod=USG_TOT.endtimeperiod AND USG_NET.aep_endtime_utc=USG_TOT.aep_endtime_utc AND 
           USG_NET.timezoneoffset=USG_TOT.timezoneoffset AND USG_NET.aep_derived_uom=USG_TOT.aep_derived_uom AND USG_NET.aep_usage_dt=USG_TOT.aep_usage_dt
    ), 
    AGG1 AS (
        SELECT MP.trsf_pole_nb, U.starttimeperiod, U.endtimeperiod, U.aep_endtime_utc, U.timezoneoffset, 
               U.aep_derived_uom, U.aep_usage_dt, 

               SUM(U.value_net) as value_net_sum, SUM(POWER(U.value_net, 2)) as value_net_sq_sum, 
               AVG(U.value_net) as value_net_mean, STDDEV_SAMP(U.value_net) as value_net_std, 
               COUNT(U.value_net) as value_net_counts, 

               SUM(U.value_rec) as value_rec_sum, SUM(POWER(U.value_rec, 2)) as value_rec_sq_sum, 
               AVG(U.value_rec) as value_rec_mean, STDDEV_SAMP(U.value_rec) as value_rec_std, 
               COUNT(U.value_rec) as value_rec_counts, 

               SUM(U.value_del) as value_del_sum, SUM(POWER(U.value_del, 2)) as value_del_sq_sum, 
               AVG(U.value_del) as value_del_mean, STDDEV_SAMP(U.value_del) as value_del_std, 
               COUNT(U.value_del) as value_del_counts, 

               SUM(U.value_tot) as value_tot_sum, SUM(POWER(U.value_tot, 2)) as value_tot_sq_sum, 
               AVG(U.value_tot) as value_tot_mean, STDDEV_SAMP(U.value_tot) as value_tot_std, 
               COUNT(U.value_tot) as value_tot_counts, 

               COUNT(*) as counts_including_null
        FROM MP
        INNER JOIN U
        ON MP.mfr_devc_ser_nbr = U.serialnumber
        WHERE U.serialnumber NOT IN ('882733737','882731886','882454685','882733663','882733768','882733694','882731883','882733767')
        GROUP BY MP.trsf_pole_nb, U.starttimeperiod, U.endtimeperiod, U.aep_endtime_utc, U.timezoneoffset, 
                 U.aep_derived_uom, U.aep_usage_dt
    )
    SELECT starttimeperiod, endtimeperiod, aep_endtime_utc, timezoneoffset, 
           aep_derived_uom, aep_usage_dt, 

           SUM(value_net_sum) AS sum_value_net_sum, AVG(value_net_sum) as mean_value_net_sum, 
           SUM(value_net_sq_sum) AS sum_value_net_sq_sum, AVG(value_net_sq_sum) as mean_value_net_sq_sum, 
           SUM(value_net_mean) AS sum_value_net_mean, AVG(value_net_mean) as mean_value_net_mean, 
           SUM(value_net_std) AS sum_value_net_std, AVG(value_net_std) as mean_value_net_std, 
           SUM(value_net_counts) AS sum_value_net_counts, AVG(value_net_counts) as mean_value_net_counts, 

           SUM(value_rec_sum) AS sum_value_rec_sum, AVG(value_rec_sum) as mean_value_rec_sum, 
           SUM(value_rec_sq_sum) AS sum_value_rec_sq_sum, AVG(value_rec_sq_sum) as mean_value_rec_sq_sum, 
           SUM(value_rec_mean) AS sum_value_rec_mean, AVG(value_rec_mean) as mean_value_rec_mean, 
           SUM(value_rec_std) AS sum_value_rec_std, AVG(value_rec_std) as mean_value_rec_std, 
           SUM(value_rec_counts) AS sum_value_rec_counts, AVG(value_rec_counts) as mean_value_rec_counts, 

           SUM(value_del_sum) AS sum_value_del_sum, AVG(value_del_sum) as mean_value_del_sum, 
           SUM(value_del_sq_sum) AS sum_value_del_sq_sum, AVG(value_del_sq_sum) as mean_value_del_sq_sum, 
           SUM(value_del_mean) AS sum_value_del_mean, AVG(value_del_mean) as mean_value_del_mean, 
           SUM(value_del_std) AS sum_value_del_std, AVG(value_del_std) as mean_value_del_std, 
           SUM(value_del_counts) AS sum_value_del_counts, AVG(value_del_counts) as mean_value_del_counts, 

           SUM(value_tot_sum) AS sum_value_tot_sum, AVG(value_tot_sum) as mean_value_tot_sum, 
           SUM(value_tot_sq_sum) AS sum_value_tot_sq_sum, AVG(value_tot_sq_sum) as mean_value_tot_sq_sum, 
           SUM(value_tot_mean) AS sum_value_tot_mean, AVG(value_tot_mean) as mean_value_tot_mean, 
           SUM(value_tot_std) AS sum_value_tot_std, AVG(value_tot_std) as mean_value_tot_std, 
           SUM(value_tot_counts) AS sum_value_tot_counts, AVG(value_tot_counts) as mean_value_tot_counts, 

           SUM(counts_including_null) AS sum_counts_including_null, AVG(counts_including_null) as mean_counts_including_null
    FROM AGG1
    GROUP BY starttimeperiod, endtimeperiod, aep_endtime_utc, timezoneoffset, 
             aep_derived_uom, aep_usage_dt
    """
    ).format(mp_where_str_3, 
             f"'{date_range[0]}'",  
             f"'{date_range[1]}'", 
             ','.join(["'{}'".format(x) for x in dev_ser_nbrs]))
    # ********************** Return **********************
    if verbose:
        print('sql_mp_1:\n', sql_mp_1, '\n\n')
        print('df_mp_1:\n', df_mp_1)
        print('sql_mp_2:\n', sql_mp_2, '\n\n')
        print('Final SQL Statement:\n', sql)
    #--------------------
    return sql